In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import pymysql

csv를 DataFrame으로 변환

In [2]:
fd_df = pd.read_csv('./data/fd210101_241030.csv')

In [3]:
print( fd_df.info() )
print( fd_df.head(3) )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      1323 non-null   int64 
 1   지점              1323 non-null   int64 
 2   지점명             1323 non-null   object
 3   일시              1323 non-null   object
 4   일 미세먼지 농도(㎍/㎥)  1323 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 51.8+ KB
None
   Unnamed: 0   지점 지점명          일시  일 미세먼지 농도(㎍/㎥)
0           0  108  서울  2021-01-01              36
1           1  108  서울  2021-01-02              43
2           2  108  서울  2021-01-03              47


In [4]:
fd_df, fd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      1323 non-null   int64 
 1   지점              1323 non-null   int64 
 2   지점명             1323 non-null   object
 3   일시              1323 non-null   object
 4   일 미세먼지 농도(㎍/㎥)  1323 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 51.8+ KB


(      Unnamed: 0   지점 지점명          일시  일 미세먼지 농도(㎍/㎥)
 0              0  108  서울  2021-01-01              36
 1              1  108  서울  2021-01-02              43
 2              2  108  서울  2021-01-03              47
 3              3  108  서울  2021-01-04              50
 4              4  108  서울  2021-01-05              37
 ...          ...  ...  ..         ...             ...
 1318        1318  108  서울  2024-10-25              29
 1319        1319  108  서울  2024-10-26              26
 1320        1320  108  서울  2024-10-27              27
 1321        1321  108  서울  2024-10-28              38
 1322        1322  108  서울  2024-10-30              28
 
 [1323 rows x 5 columns],
 None)

In [ ]:
# 미세먼저 
# 0~30 좋음
# 31~80 보통
# 81~150 나쁨
# 151이상 매우나쁨

# DB 저장

In [5]:
conn = pymysql.connect(host="localhost", port=3306, db="mp",
                       user="root", passwd="1234")

# 2. 명령 도구 준비
cursor = conn.cursor()

In [6]:
# sql = "drop table fine_dust"
# cursor.execute(sql)
create_table_query = """
CREATE TABLE IF NOT EXISTS fine_dust (
    station_id INT,
    station_name VARCHAR(50),
    date VARCHAR(30),
    `fine_dust(㎍/㎥)` INT
)
"""
cursor.execute(create_table_query)

0

In [7]:
# DataFrame의 각 행을 MySQL에 삽입
for _, row in fd_df.iterrows():
    insert_query = """
    INSERT INTO fine_dust (station_id, station_name, date,`fine_dust(㎍/㎥)`)
    VALUES (%s, %s, %s, %s)
    """

    # np.nan을 None으로 변환하는 함수
    def convert_nan(value):
        return None if isinstance(value, float) and np.isnan(value) else value
    

    values = (
        convert_nan(row['지점']),
        convert_nan(row['지점명']),
        convert_nan(row['일시']),
        convert_nan(row['일 미세먼지 농도(㎍/㎥)'])
    )
    cursor.execute(insert_query, values)


In [8]:
conn.commit()


In [9]:
conn.close()
cursor.close()

In [16]:
# db 저장 완료